In [2]:
import os
import matplotlib.pyplot as plt
import scipy

In [1]:
CAMEO_PREDICTION_PATH = "../data/taccruns/predictions-cameo/"
CAMEO_PREDICTION_PATH_ZEROED_SINGLE_REP = "../data/taccruns/predictions-cameo-mod-s0/"
FASTA_DIR = "../data/taccruns/fasta_dir/"

In [ ]:
def tmscore(target, modified):
    tmscore = -1
    os.system(f"./tmscore/TMscore {target} {modified} >> tmscore_temp_output")
    with open("tmscore_temp_output") as f:
        for line in f:
            if line.startswith("TM-score"):
                tmscore = float(line.split('=')[1].strip().split(' ')[0])
    os.system("rm tmscore_temp_output")
    return tmscore
def get_amino_acid_len(protien):
    with open(FASTA_DIR + protien + '.fasta') as f:
        fastafile = f.readlines()
    return(len(fastafile[1]))

get_amino_acid_len('fasta_dir/','7rdt_A')

In [ ]:
effn_dict = {}
with open() as f:
    lines = f.readlines()
for line in lines:
    effn_dict[line.split()[0]] = float(line.split()[2])

In [ ]:
cameo_dict = {}
for pdb_file in os.listdir(CAMEO_PREDICTION_PATH): #for each predicted structure
    if pdb_file[-1] == "b": #check if is a pdb file
        if pdb_file[19:26] == 'unrelax': #check if it the unrelaxed prediction
            protein = pdb_file[0:6]
            cameo_dict[protein] = {}
            cameo_dict[protein]['tmscore'] = tmscore(CAMEO_PREDICTION_PATH + protein, CAMEO_PREDICTION_PATH_ZEROED_SINGLE_REP + protein)
            cameo_dict[protein]['effn'] = effn_dict[protein]
            cameo_dict[protein]['seqlen'] = get_amino_acid_len(protein)

In [ ]:

plt.rcParams['figure.figsize'] = [15, 6]
for protein in cameo_dict.keys():
    plt.scatter(protein, cameo_dict[protein]['tmscore'])
plt.legend()
plt.ylabel('tmscore')
plt.title('tmscore between regular and modified (s=s*0) pdb')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
for prop in ['effn', 'seqlen']:
    x=[]
    y=[]
    for protein in cameo_dict.keys():
        plt.scatter(cameo_dict[protein][prop], cameo_dict[protein]['tmscore'])
        x.append(cameo_dict[protein][prop])
        y.append(cameo_dict[protein]['tmscore'])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    plt.title('tmscore between regular and modified (s=s*0) pdb \n r^2 {:.4}'.format(r_value**2))
    plt.xlabel(prop)
    plt.ylabel('tmscore')
    plt.show()